In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os
import numpy.linalg as la
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import sys
import pandas as pd
#sys.path.append("/home/CVShare/Jeni/hts/machine_learning/sampling_with_data_cleaning")
import sampling_with_data_cleaning as sdc
from utils import *

### FNN on cleaned features

In [2]:
findex = 2
x,y = get_features(findex,train=True,cleaned=True)
print y.shape
print sum(y)
y = y.astype(int)
print sum(y)
x,y = shuffle(x,y)

(7881,)
129.0
129


In [3]:
no_ex,ip_dim = x.shape
print no_ex,ip_dim

7881 153


In [4]:
class c_mlp(nn.Module):
    def __init__(self):
        super(c_mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim,1000)
        self.l2 = nn.Linear(1000,1000)
        self.l3 = nn.Linear(1000,500)
        #self.l4 = nn.Linear(500,500)
        self.l5 = nn.Linear(500,100)
        self.l6 = nn.Linear(100,50)
        self.l7 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.leaky_relu(self.l3(x))
        #x = F.leaky_relu(self.l4(x))
        x = F.leaky_relu(self.l5(x))
        x = F.leaky_relu(self.l6(x))
        x = (self.l7(x))
        
        return x

In [5]:
max_percent = 0.6
model_no = 1
for i in range(1,6):
    val_iter = i
    print("val iter: ",val_iter)
    
   
    #weights_array = [7]
    weights_array = np.linspace(30,50,5)
    
    for i,w in enumerate(weights_array): 
        mymlp = c_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-3)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(2500):
            
            ind = get_train_ind(val_iter=val_iter,no_examples=no_ex)
            xtrain,ytrain = get_train_batch(x,y,batch_size=100,indices=ind)
            
            model_op = mymlp(xtrain)
           
            loss = criterion(model_op,ytrain)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    #     if(ep%30==29):
    #         print(loss.data[0])

        ## After training check on cross validation data
        xval,yval = get_val_data(x,y,no_examples=no_ex,val_iter=val_iter)
        min_fp = xval.size()[0]*0.2
        yval = yval.reshape(yval.shape[0],)
        train_op = mymlp(xval)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
      
        cf = metrics.confusion_matrix(yval,pred_labels).ravel()
        [tn,fp,fn,tp] = cf
        print('tn, fp, fn, tp: ',cf)
        percent = float(float(tp)/float(tp+fn))
        if(percent>max_percent*0.98):
            if(fp<min_fp):
                if(model_no<6):
                        max_percent = percent
                        print("min fp, max_percent",fp,percent)
                        model_path = os.getcwd() + '/fnn_clean_' + str(findex)+'_model'+str(model_no)
                        torch.save(mymlp.state_dict(),model_path)
                        print("saving model on val: ",val_iter," and weight: ",w)
                        model_no = model_no + 1
                else:
                    break
        


('val iter: ', 1)
('tn, fp, fn, tp: ', array([1545,    6,   10,   15]))
('min fp, max_percent', 6, 0.6)
('saving model on val: ', 1, ' and weight: ', 30.0)
('tn, fp, fn, tp: ', array([1545,    6,   10,   15]))
('min fp, max_percent', 6, 0.6)
('saving model on val: ', 1, ' and weight: ', 35.0)
('tn, fp, fn, tp: ', array([1545,    6,   12,   13]))
('tn, fp, fn, tp: ', array([1543,    8,   10,   15]))
('min fp, max_percent', 8, 0.6)
('saving model on val: ', 1, ' and weight: ', 45.0)
('tn, fp, fn, tp: ', array([1538,   13,   10,   15]))
('min fp, max_percent', 13, 0.6)
('saving model on val: ', 1, ' and weight: ', 50.0)
('val iter: ', 2)
('tn, fp, fn, tp: ', array([1544,    3,   16,   13]))
('tn, fp, fn, tp: ', array([1544,    3,   14,   15]))
('tn, fp, fn, tp: ', array([1542,    5,   16,   13]))
('tn, fp, fn, tp: ', array([1544,    3,   18,   11]))
('tn, fp, fn, tp: ', array([1541,    6,   15,   14]))
('val iter: ', 3)
('tn, fp, fn, tp: ', array([1544,    9,   12,   11]))
('tn, fp, fn, t

KeyboardInterrupt: 

In [6]:
xtest_t,ytest = get_features(findex,train=False)
xtest = Variable(torch.cuda.FloatTensor(xtest_t).cuda())
final_pred = np.zeros(ytest.shape[0],)
print final_pred.shape
for i in range(1,6):
    model_path = os.getcwd() + '/fnn_clean_' + str(findex)+'_model'+str(i)
    
    
    test_model = c_mlp().cuda()
    test_model.load_state_dict(torch.load(model_path))
    test_op = test_model(xtest)
    test_op = test_op.cpu().data.numpy()
    pred_labels = np.argmax(test_op,axis=1).reshape(final_pred.shape[0],)
    print pred_labels.shape
    final_pred = pred_labels + final_pred
    print final_pred.shape
    cf = metrics.confusion_matrix(ytest,pred_labels).ravel()
    print('tn, fp, fn, tp: ',cf)

(1996,)
(1996,)
(1996,)
('tn, fp, fn, tp: ', array([1979,   12,    5,    0]))
(1996,)
(1996,)
('tn, fp, fn, tp: ', array([1975,   16,    5,    0]))
(1996,)
(1996,)
('tn, fp, fn, tp: ', array([1969,   22,    5,    0]))
(1996,)
(1996,)
('tn, fp, fn, tp: ', array([1973,   18,    5,    0]))
(1996,)
(1996,)
('tn, fp, fn, tp: ', array([1976,   15,    5,    0]))


In [ ]:
final_pred[final_pred>1] = 1
cf = metrics.confusion_matrix(ytest,final_pred).ravel()
print('tn, fp, fn, tp: ',cf)

In [ ]:
pred_labels.shape

In [ ]:
final_pred.shape